In [1]:
"""
inference_openai.py - text generation with OpenAI API
    See https://platform.openai.com/docs/quickstart for more details.
Usage:
python inference_openai.py --prompt "The quick brown fox jumps over the lazy dog." --model "gpt-3.5-turbo" --temperature 0.5 --max_tokens 256 --n 1 --stop "."
Detailed usage:
python inference_openai.py --help
Notes:
- The OpenAI API key can be set using the OPENAI_API_KEY environment variable (recommended) or using the --api_key argument.
- This script supports inference with the "chat" models only.
"""

import json
import logging
import os
from datetime import datetime
from pathlib import Path
from typing import List, Optional, Union

import fire
import openai
from cleantext import clean
from tqdm.auto import tqdm

AVAILABLE_MODELS = [
    "gpt-4",
    "gpt-4-0314",
    "gpt-4-32k",
    "gpt-4-32k-0314",
    "gpt-3.5-turbo",
    "gpt-3.5-turbo-0301",
    "text-davinci-003",
    "code-davinci-002",
]

logging.basicConfig(
    format="%(asctime)s | %(levelname)s: %(message)s",
    datefmt="%b/%d %H:%M:%S",
    level=logging.INFO,
)


def validate_model(model):
    """
    Validates the given model name against the list of available models (see AVAILABLE_MODELS)
        NOTE: this does not mean you have access to the model, just a basic check.
    :param model: The name of the model to validate.
    :raises ValueError: If the given model is not in the list of available models.
    """
    if model not in AVAILABLE_MODELS:
        raise ValueError(
            f"Invalid model '{model}', available models: {', '.join(AVAILABLE_MODELS)}"
        )


def chat_generate_text(
    prompt: str,
    openai_api_key: str = None,
    model: str = "gpt-3.5-turbo",
    system_prompt: str = "You are a helpful assistant.",
    temperature: float = 0.5,
    max_tokens: int = 256,
    n: int = 1,
    stop: Optional[Union[str, list]] = None,
    presence_penalty: float = 0,
    frequency_penalty: float = 0.1,
) -> List[str]:
    """
    chat_generate_text - Generates text using the OpenAI API.
    :param str prompt: prompt for the model
    :param str openai_api_key: api key for the OpenAI API, defaults to None
    :param str model: model to use, defaults to "gpt-3.5-turbo"
    :param str system_prompt: initial prompt for the model, defaults to "You are a helpful assistant."
    :param float temperature: _description_, defaults to 0.5
    :param int max_tokens: _description_, defaults to 256
    :param int n: _description_, defaults to 1
    :param Optional[Union[str, list]] stop: _description_, defaults to None
    :param float presence_penalty: _description_, defaults to 0
    :param float frequency_penalty: _description_, defaults to 0.1
    :return List[str]: _description_
    """
    if openai_api_key is None:
        openai_api_key = os.environ.get("OPENAI_API_KEY", None)
    assert openai_api_key is not None, "OpenAI API key not found."

    openai.api_key = openai_api_key

    messages = [
        {"role": "system", "content": f"{system_prompt}"},
        {"role": "user", "content": prompt},
    ]

    response = openai.ChatCompletion.create(
        model=model,
        messages=messages,
        temperature=temperature,
        max_tokens=max_tokens,
        n=n,
        stop=stop,
        presence_penalty=presence_penalty,
        frequency_penalty=frequency_penalty,
    )

    generated_texts = [
        choice.message["content"].strip() for choice in response["choices"]
    ]
    return generated_texts



def get_timestamp():
    """Returns the current timestamp in the format YYYYMMDD_HHMM"""
    return datetime.now().strftime("%Y%b%d_%H-%M")


def read_and_clean_file(file_path, lower=False):
    """
    Reads the content of a file and cleans the text using the cleantext package.
    :param file_path: The path to the file.
    :return: The cleaned text.
    """
    with open(file_path, "r", encoding="utf-8") as f:
        context = clean(f.read(), lower=lower)
    return context


def save_output_to_file(
    out_dir,
    output,
    file_name,
):
    """
    Saves the generated output to a file.
    :param out_dir: The output directory.
    :param output: The text to be saved.
    :param file_name: The name of the output file.
    """
    out_dir = Path(out_dir)
    out_dir.mkdir(parents=True, exist_ok=True)
    output_file = out_dir / file_name

    with output_file.open("w") as f:
        f.write(output)


def generate_text(
    prompt: Optional[str] = None,
    api_key: Optional[str] = None,
    model: str = "gpt-3.5-turbo",
    system_prompt: str = "You are a helpful assistant.",
    temperature: float = 0.5,
    max_tokens: int = 256,
    n: int = 1,
    stop: Optional[Union[str, list]] = None,
    presence_penalty: float = 0.0,
    frequency_penalty: float = 0.0,
):
    """
    Main function to run the text generation script.
    :param prompt: The input prompt for the model.
    :param api_key: The OpenAI API key. If not provided, checks the environment variable OPENAI_API_KEY.
    :param model: openai model code, defaults to "gpt-3.5-turbo"
    :param system_prompt: The system prompt for the model, defaults to "You are a helpful assistant."
    :param temperature: The sampling temperature (creativity) for the model. (default: 0.5)
    :param max_tokens: The maximum number of tokens in the generated text. (default: 256)
    :param n: The number of generated texts. (default: 1)
    :param stop: The stopping sequence(s) for the model. (default: None)
    :param presence_penalty: The penalty applied for new token presence. (default: 0.0)
    :param frequency_penalty: The penalty applied based on token frequency. (default: 0.0)
    :param file_path: The path to a file/directory to include after the prompt.
    :param out_dir: directory to save outputs. (default: parent directory of input path if provided)
    :param save_prompt: Save the input prompt in the output files with the generated text. (default: False)
    :param markdown: save the generated text as a markdown file. (default: False)
    :param verbose: Whether to print the generated text to the console.
    """
    openai.api_key = api_key if api_key else os.getenv("OPENAI_API_KEY")
    assert (
        openai.api_key is not None
    ), "API key not found - pass as arg or set environment variable OPENAI_API_KEY"

    prompt = prompt.strip()
    validate_model(model)

    generated_texts = chat_generate_text(
        prompt=prompt,
        model=model,
        system_prompt=system_prompt,
        temperature=temperature,
        max_tokens=max_tokens,
        n=n,
        stop=stop,
        presence_penalty=presence_penalty,
        frequency_penalty=frequency_penalty,
    )

    return generated_texts

            

ModuleNotFoundError: No module named 'cleantext'

In [ ]:
text = ''
print(f"Generated text: {generated_texts}")

In [3]:
import re

def parse_and_trim(text):
    # Remove JavaScript code
    text_without_js = re.sub(r'\/\/.*?(\n|$)', '', text, flags=re.DOTALL)
    
    # Trim leading and trailing whitespaces
    trimmed_text = text_without_js.strip()
    
    return trimmed_text

# Example text
vietnamese_text_with_js = """
QKKT Vân Phong cũng được định hướng để trở thành khu vực phát triển đô thị thông minh bền vững với hệ thống cơ sở hạ tầng kinh tế - xã hội phát triển đồng bộ, hiện đại; có vị trí quan trọng về an ninh, quốc phòng của quốc gia.Đáng chú ý, theo quy hoạch, khu vực dự kiến phát triển cảng hàng không khoảng 500 ha tại xã Vạn Thắng, H.Vạn Ninh - khu vực phía bắc Vân Phong. Các khu vực phát triển sân golf được bố trí tại đảo Hòn Lớn, khu Đầm Môn, khu vực Tuần Lễ - Hòn Ngang.Mới đây, tại cuộc họp góp ý hoàn chỉnh Đề án quy hoạch Cảng hàng không Vân Phong, Chủ tịch UBND tỉnh Khánh Hòa Nguyễn Tấn Tuân đã giao Sở GTVT phối hợp với đơn vị tư vấn khẩn trương hoàn chỉnh đề án.Ông Tuân cũng đề nghị đơn vị tư vấn phải làm rõ được nguồn cung cấp vật liệu phục vụ dự án; bổ sung kho chứa xăng dầu trong cảng theo quy mô, thẩm quyền phê duyệt; bổ sung kho chứa hàng hóa, các khu vực tác nghiệp, sửa chữa máy bay, đồng thời đánh giá, làm rõ hơn hiệu quả đầu tư dự án.
    //Chèn ads giữa bài
    (runinit = window.runinit || []).push(function () {
        //Nếu k chạy ads thì return
        if (typeof _chkPrLink != 'undefined' && _chkPrLink)
            return;

        var mutexAds = '<zone id="l2srqb41"></zone>';
        var content = $('[data-role="content"]');
        if (content.length > 0) {
            var childNodes = content[0].childNodes;
            for (i = 0; i < childNodes.length; i++) {
                var childNode = childNodes[i];

                var isPhotoOrVideo = false;
                if (childNode.nodeName.toLowerCase() == 'div') {
                    // kiem tra xem co la anh khong?
                    var type = $(childNode).attr('class') + '';

                    if (type.indexOf('VCSortableInPreviewMode') >= 0) {
                        isPhotoOrVideo = true;
                    }
                }

                try {
                    if ((i >= childNodes.length / 2 - 1) && (i < childNodes.length / 2) && !isPhotoOrVideo) {
                        if (i <= childNodes.length - 3) {
                            childNode.after(htmlToElement(mutexAds));
                            arfAsync.push("l2srqb41");
                        }
                        break;
                    }
                }
                catch (e) { }
            }
        }
    });
"""

# Parse and trim the text
trimmed_text = parse_and_trim(vietnamese_text_with_js)

# Print the result
print(trimmed_text)


QKKT Vân Phong cũng được định hướng để trở thành khu vực phát triển đô thị thông minh bền vững với hệ thống cơ sở hạ tầng kinh tế - xã hội phát triển đồng bộ, hiện đại; có vị trí quan trọng về an ninh, quốc phòng của quốc gia.Đáng chú ý, theo quy hoạch, khu vực dự kiến phát triển cảng hàng không khoảng 500 ha tại xã Vạn Thắng, H.Vạn Ninh - khu vực phía bắc Vân Phong. Các khu vực phát triển sân golf được bố trí tại đảo Hòn Lớn, khu Đầm Môn, khu vực Tuần Lễ - Hòn Ngang.Mới đây, tại cuộc họp góp ý hoàn chỉnh Đề án quy hoạch Cảng hàng không Vân Phong, Chủ tịch UBND tỉnh Khánh Hòa Nguyễn Tấn Tuân đã giao Sở GTVT phối hợp với đơn vị tư vấn khẩn trương hoàn chỉnh đề án.Ông Tuân cũng đề nghị đơn vị tư vấn phải làm rõ được nguồn cung cấp vật liệu phục vụ dự án; bổ sung kho chứa xăng dầu trong cảng theo quy mô, thẩm quyền phê duyệt; bổ sung kho chứa hàng hóa, các khu vực tác nghiệp, sửa chữa máy bay, đồng thời đánh giá, làm rõ hơn hiệu quả đầu tư dự án.
        (runinit = window.runinit || []).p